In [1]:
import pandas as pd 
import os 
import glob

In [2]:


def read_and_adjust_word_list(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file, header=None)
    
    # Adjust the numbering to start from 1
    word_list = df[0].tolist()
    
    # Return the adjusted word list
    return word_list

def rename_file(filepath, new_name):
    folder, old_name = os.path.split(filepath)
    new_path = os.path.join(folder, new_name)
    os.rename(filepath, new_path)

def rename_files_in_folder(folder, start_index, word_list, specific_start, kinect_studio_extension):
    # Get all files in the folder that start with the specific string
    files = glob.glob(os.path.join(folder, f'{specific_start}*.{kinect_studio_extension}'))
    
    # Sort files by creation time
    files.sort(key=os.path.getctime)
    
    # Rename files according to the word list
    for i, file in enumerate(files):
        if start_index + i < len(word_list):
            new_name = f'P55W{start_index + i + 1}' + os.path.splitext(file)[1]  # Keep the original file extension
            new_path = os.path.join(folder, new_name)
            try:
                os.rename(file, new_path)
                print(f'Renamed {file} to {new_name}')
            except Exception as e:
                print(f"Error renaming {file}: {e}")

def main():
    # Path to the root directory containing the folders
    root_dir = 'E:\ISL_DATASET\FIRST FIFTY'

    # Path to the CSV word list file
    csv_file = 'E:\Words_list.csv'
    
    # File extension for kinect studio format
    kinect_studio_extension = 'xef'  # replace with the actual extension
    
    # Specific start string
    specific_start = '20240704_06'
    
    # Read and adjust the word list
    word_list = read_and_adjust_word_list(csv_file)
    
    # Get all folders in the root directory
    folders = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
    
    # Sort folders according to the word list
    sorted_folders = sorted(folders, key=lambda x: word_list.index(os.path.basename(x)))
    
    # Rename files in the folders according to the word list
    rename_files_in_folder(sorted_folders[0], 0, word_list, specific_start, kinect_studio_extension)
    rename_files_in_folder(sorted_folders[1], 50, word_list, specific_start, kinect_studio_extension)

if __name__ == "__main__":
    main()


In [2]:
import os
import cv2
import csv
import mediapipe as mp

# Set TensorFlow environment variable to disable oneDNN custom operations warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=2, smooth_landmarks=True,
                                enable_segmentation=False, refine_face_landmarks=True, 
                                min_detection_confidence=0.5, min_tracking_confidence=0.5)

def generate_headers():
    headers = []
    for i in range(33):
        headers.extend([f'pose_{i}_x', f'pose_{i}_y', f'pose_{i}_z', f'pose_{i}_visibility'])
    for i in range(21):
        headers.extend([f'left_hand_{i}_x', f'left_hand_{i}_y', f'left_hand_{i}_z'])
    for i in range(21):
        headers.extend([f'right_hand_{i}_x', f'right_hand_{i}_y', f'right_hand_{i}_z'])
    return headers

def process_video(video_path, output_csv_path):
    print("RUNNING")
    cap = cv2.VideoCapture(video_path)
    all_landmarks = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)
        landmarks = []
        if results.pose_landmarks:
            for landmark in results.pose_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        else:
            landmarks.extend([0, 0, 0, 0] * 33)
        if results.left_hand_landmarks:
            for landmark in results.left_hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])
        else:
            landmarks.extend([0, 0, 0] * 21)
        if results.right_hand_landmarks:
            for landmark in results.right_hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])
        else:
            landmarks.extend([0, 0, 0] * 21)
        all_landmarks.append(landmarks)
    cap.release()
    print("running")
    headers = generate_headers()
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(all_landmarks)
    print(f'Saved landmarks to {output_csv_path}')

dataset_path = "E:\\ISL_DATASET\\POSTPROCESSED\\VIDEOS\\COLOR\\TEA\\W1P1.mp4"
processed_data_path = "E:\\ISL_DATASET\\Output CSV Files\\W1P1.csv"
process_video(dataset_path, processed_data_path)


RUNNING
running
Saved landmarks to E:\ISL_DATASET\Output CSV Files\W1P1.csv


In [1]:
import pandas as pd

In [6]:
df= pd.read_csv("E:\ISL_DATASET\Output CSV Files\W1P1.csv")

In [7]:
df.head(10)

,pose_0_x,pose_0_y,pose_0_z,pose_0_visibility,pose_1_x,pose_1_y,pose_1_z,pose_1_visibility,pose_2_x,pose_2_y,...,right_hand_17_z,right_hand_18_x,right_hand_18_y,right_hand_18_z,right_hand_19_x,right_hand_19_y,right_hand_19_z,right_hand_20_x,right_hand_20_y,right_hand_20_z
0,0.524480,0.577721,-0.460696,0.999988,0.526493,0.546685,-0.435589,0.999960,0.530713,0.545987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.524473,0.577669,-0.475680,0.999986,0.526483,0.546693,-0.449455,0.999948,0.530709,0.545986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.524426,0.577644,-0.482522,0.999980,0.526462,0.546692,-0.455955,0.999926,0.530698,0.545971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.524393,0.577649,-0.502905,0.999976,0.526434,0.546740,-0.476516,0.999910,0.530652,0.545986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.524386,0.577697,-0.500390,0.999971,0.526412,0.546891,-0.474363,0.999893,0.530611,0.546083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.524374,0.577867,-0.495660,0.999965,0.526388,0.547272,-0.469991,0.999869,0.530578,0.546410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.522868,0.578437,-0.490924,0.999961,0.525932,0.548051,-0.464586,0.999852,0.530477,0.547231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.516123,0.580483,-0.491864,0.999956,0.523289,0.550688,-0.465919,0.999836,0.528805,0.550141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.512287,0.582215,-0.491173,0.999953,0.521445,0.552665,-0.465526,0.999823,0.527310,0.552222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.511137,0.583243,-0.492514,0.999947,0.520603,0.553856,-0.466898,0.999803,0.526599,0.553428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import pandas as pd

In [6]:
df_lsa = pd.read_csv(r"E:\ISL_DATASET\SPOTER\normalized CSV\normalized_LSA64_60fps.csv")
df_mediapipe = pd.read_csv(r"E:\ISL_DATASET\Output CSV Files\W1P1.csv")

In [7]:
print(f" LSA file shape {df_lsa.shape}")
print(f" MediaPipe file shape {df_mediapipe.shape}")

 LSA file shape (3177, 112)
 MediaPipe file shape (65, 258)


In [9]:
count_x = 0
count_y = 0


for i in df_lsa.columns:
    if i.endswith("X"):
        count_x += 1


for i in df_lsa.columns:
    if i.endswith("Y"):
        count_y += 1

In [10]:
print(f"X: {count_x}")
print(f"Y :{count_y}")

X: 55
Y :55


In [13]:
for i in df_lsa.columns:
    if i.endswith("X") or i.endswith("Y"):
        continue 
    else:
        print(i)

labels
video_fps


In [12]:
df_mediapipe.columns

Index(['pose_0_x', 'pose_0_y', 'pose_0_z', 'pose_0_visibility', 'pose_1_x',
       'pose_1_y', 'pose_1_z', 'pose_1_visibility', 'pose_2_x', 'pose_2_y',
       ...
       'right_hand_17_z', 'right_hand_18_x', 'right_hand_18_y',
       'right_hand_18_z', 'right_hand_19_x', 'right_hand_19_y',
       'right_hand_19_z', 'right_hand_20_x', 'right_hand_20_y',
       'right_hand_20_z'],
      dtype='object', length=258)

In [ ]:
df_mediapipe[df_mediapipe[""]]

In [13]:
df_mediapipe.head()

,pose_0_x,pose_0_y,pose_0_z,pose_0_visibility,pose_1_x,pose_1_y,pose_1_z,pose_1_visibility,pose_2_x,pose_2_y,...,right_hand_17_z,right_hand_18_x,right_hand_18_y,right_hand_18_z,right_hand_19_x,right_hand_19_y,right_hand_19_z,right_hand_20_x,right_hand_20_y,right_hand_20_z
0,0.524480,0.577721,-0.460696,0.999988,0.526493,0.546685,-0.435589,0.999960,0.530713,0.545987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.524473,0.577669,-0.475680,0.999986,0.526483,0.546693,-0.449455,0.999948,0.530709,0.545986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.524426,0.577644,-0.482522,0.999980,0.526462,0.546692,-0.455955,0.999926,0.530698,0.545971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.524393,0.577649,-0.502905,0.999976,0.526434,0.546740,-0.476516,0.999910,0.530652,0.545986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.524386,0.577697,-0.500390,0.999971,0.526412,0.546891,-0.474363,0.999893,0.530611,0.546083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_lsa.head()

,middlePIP_0_Y,littlePIP_1_Y,middleTip_1_Y,thumbIP_0_X,middleMCP_1_Y,indexMCP_1_X,middleTip_1_X,middlePIP_1_Y,rightElbow_X,indexTip_0_Y,...,indexDIP_1_X,littleDIP_1_X,littleMCP_1_Y,thumbCMC_1_X,ringDIP_1_X,wrist_0_X,indexPIP_0_Y,leftEye_Y,labels,video_fps
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.6078344821341666, 0.7239028250060661, 0.723...","[0.33543993978699066, 0.4275415458044989, 0.42...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.32754547100834136, 0.28119420875567136, 0.2...","[0.5614191085758263, 0.4971889360869991, 0.497...","[0.6061049143775536, 0.6162223284481478, 0.616...","[0.21332326010435462, 0.29786141862309506, 0.2...","[0.2790096616775772, 0.2784623108390339, 0.278...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.6662330367955813, 0.703786290337183, 0.7037...","[0.7024796837685191, 0.6925148746473724, 0.692...","[0.6813110477289962, 0.7024916618683931, 0.702...","[0.284568011272962, 0.256053067092536, 0.25605...","[0.6936797562052222, 0.7039492505640232, 0.703...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.7975356462121056, 0.8100301569602382, 0.810...",24,60
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.8636583561020035, 0.8636583561020035, 0.916...","[0.458909665300546, 0.458909665300546, 0.49246...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.4694472905282328, 0.4694472905282328, 0.472...","[0.5624110598816028, 0.5624110598816028, 0.571...","[0.6524398053278685, 0.6524398053278685, 0.650...","[0.37461577868852414, 0.37461577868852414, 0.4...","[0.22749684324907857, 0.22749684324907857, 0.2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.6405360598816028, 0.6405360598816028, 0.718...","[0.7382421163479053, 0.7382421163479053, 0.665...","[0.9166666666666666, 0.9166666666666666, 0.866...","[0.2190595116120216, 0.2190595116120216, 0.263...","[0.7526575307377047, 0.7526575307377047, 0.709...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.7777015974139527, 0.7777015974139527, 0.776...",24,60
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.8929411671197164, 0.912279218200803, 0.9166...","[0.5287395149453423, 0.5581949693859043, 0.547...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.528275867550511, 0.5668350564485687, 0.5370...","[0.5820457182825439, 0.5631185934673586, 0.569...","[0.6508046269360591, 0.6400827732596045, 0.631...","[0.46400109155843816, 0.5021447024623634, 0.55...","[0.22726099705671138, 0.22669129074878572, 0.2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.7072371384212672, 0.6950300503453583, 0.686...","[0.7130790955961448, 0.6788589937791372, 0.611...","[0.9166666666666665, 0.9166666666666667, 0.853...","[0.24988210109102849, 0.25735142726885, 0.2950...","[0.7337842635424779, 0.7146326308513611, 0.675...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.7773613833202373, 0.7767020220210338, 0.776...",24,60
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.9166666666666667, 0.9166666666666666, 0.846...","[0.5782471983882285, 0.5667680677954823, 0.627...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.5594785987542672, 0.5622988384625482, 0.514...","[0.5951008537325857, 0.58950706295374, 0.53167...","[0.6438702472370088, 0.6351514361616576, 0.659...","[0.5153866041869158, 0.5854260603674103, 0.600...","[0.22290431793910095, 0.22285504265565312, 0.2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.6940181860703625, 0.6980256821236374, 0.694...","[0.6794656822672941, 0.6167337744328654, 0.526...","[0.8849708520804771, 0.8594448172653678, 0.723...","[0.3081649576995778, 0.3210436459298124, 0.337...","[0.7061944424776481, 0.6642638204100044, 0.607...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.773994

In [21]:
df_lsa["labels"].unique()

array([24, 23, 15, 12, 46, 41, 48, 13, 14, 22, 25, 49, 40, 47, 62, 54, 53,
       38,  7,  9, 36, 31, 52, 55, 63, 64, 30,  8, 37,  1, 39,  6, 42, 45,
       20, 27, 18, 11, 16, 29, 44, 43, 17, 28, 10, 26, 19, 21,  3,  4, 32,
       35, 61, 59, 50, 57, 34, 33,  5,  2, 56, 51, 58, 60], dtype=int64)

c:\Users\iiit sricity2\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
